In [ ]:
import json
import numpy as np
from utils.transform_utils import quat2mat, convert_quat
import cv2
from scipy.spatial.transform import Rotation

import sys
sys.path.append("/home/vdrame/catkin_ws/src/py_panda/PyPanda")
from PyPanda import Robot
import rospy

from utils.camera_utils import RealCamera, RealCameraROS

In [ ]:
rospy.init_node("pythonnode_register_views",anonymous=True)
rbt = Robot("panda", use_gripper=True)

camera_connexion = "ROS"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    #retrieve image and depth
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()

    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    intrinsic, distortion = camera.getIntrinsic()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")

# views_pos = []
# views_orn_wxyz = []
views = {}
view_idx = 0
while True:
    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    cv2.imshow("rgb", rgb)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('s'):
        view_pos = rbt.model.ee_pos_rel()
        view_orn_wxyz = rbt.model.ee_orn_rel()
        views["view"+view_idx] = {"pos": view_pos, "orn_wxyz": view_orn_wxyz}
        view_idx += 1
        print("view{} saved",format(view_idx))

print(views)

#write pos in json file
with open('config/views_pos.json', 'w') as outfile:
    json.dump(views, outfile)

In [41]:
#load pos and orn from json file
with open('config/views_pos.json') as json_file:
    views_pos_1 = json.load(json_file)

keys = views_pos_1.keys()
for key in keys:
    print(key)
    print(views_pos_1[key]["pos"])
    print(views_pos_1[key]["orn_wxyz"])
    print("\n")

view0
[0.36072976, 0.41291873, 0.30648647]
[0.1655683, 0.81494402, 0.28065989, -0.47925298]


view1
[0.10118165, 0.47394513, 0.30265239]
[0.3190109, 0.94507174, -0.07051679, 0.00994117]


view2
[0.00465632, 0.32838561, 0.26232907]
[0.06700862, 0.96685827, -0.02494526, 0.24509727]




# Write camera extrinsic

In [37]:
#D415
D415_hand2camera_pos = np.array([0.0488546636437146,-0.03384417860749521,0.0512776975002817]) 
D415_hand2camera_quat = [0.012961267509189803,-0.0012768531849757236,0.7052247395136084,0.708864191484139] #xyzw 
D415_intrinsic = np.array([[925,    0.,           626],
                           [0.,         926,      345],
                           [0.,         0.,           1.]])
#L515
L515_hand2camera_pos = np.array([0.08329189218278059, 0.0014213145240625528, 0.0504764049956106]) 
L515_hand2camera_quat = [0.01521805627198811, 0.00623363612254646, 0.712108725756912, 0.7018765669580811] #xyzw 
L515_intrinsic = np.array([[915.73510742, 0.,           650.41772461],
                           [0.,         916.14764404, 356.6946106],
                           [0.,         0.,           1.]])

camera_calibration = {}
camera_calibration["D415"] = {"pos": D415_hand2camera_pos.tolist(), "quat_xyzw": D415_hand2camera_quat, "intrinsic": D415_intrinsic.tolist()}
camera_calibration["L515"] = {"pos": L515_hand2camera_pos.tolist(), "quat_xyzw": L515_hand2camera_quat, "intrinsic": L515_intrinsic.tolist()}
camera_calibration
#write in a json file
with open('config/camera_calibration.json', 'w') as outfile:
    json.dump(camera_calibration, outfile)

In [38]:
#read from camera calin from json file
with open('config/camera_calibration.json') as json_file:
    camera_calibration_read = json.load(json_file)

print(camera_calibration_read["D415"]["pos"])
print(camera_calibration_read["D415"]["quat_xyzw"])
print(camera_calibration_read["D415"]["intrinsic"])

[0.0488546636437146, -0.03384417860749521, 0.0512776975002817]
[0.012961267509189803, -0.0012768531849757236, 0.7052247395136084, 0.708864191484139]
[[925.0, 0.0, 626.0], [0.0, 926.0, 345.0], [0.0, 0.0, 1.0]]


In [ ]:
# view_init_pos = [0.17908377, 0.23450877, 0.44783125]
# view_init_orn_wxyz = [0.0030477, 0.92571715, 0.37800879, 0.01215875]
# view_front_pos = [0.36072976, 0.41291873, 0.30648647]
# view_front_orn_wxyz = [ 0.1655683,   0.81494402,  0.28065989, -0.47925298]
# view_right_pos = [0.10118165, 0.47394513, 0.30265239]
# view_right_orn_wxyz = [ 0.3190109,   0.94507174, -0.07051679,  0.00994117]
# view_back_pos = [0.00465632, 0.32838561, 0.26232907]
# view_back_orn_wxyz = [ 0.06700862,  0.96685827, -0.02494526,  0.24509727]

# views_pos = [view_front_pos, view_right_pos, view_back_pos]
# views_orn_wxyz = [view_front_orn_wxyz, view_right_orn_wxyz, view_back_orn_wxyz]

# views = {}
# for i in range(len(views_pos)):
#     views["view"+str(i)] = {"pos": views_pos[i], "orn_wxyz": views_orn_wxyz[i]}
# print(views)